In [1]:
import gymnasium as gym
import numpy as np
from typing import Any, Dict, List, Tuple, Optional
from ray.rllib.env.multi_agent_env import MultiAgentEnv

2023-08-14 19:24:01,850	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
from ray.tune.registry import register_env
from ray.rllib.algorithms import ppo, dqn, appo, impala
from ray.rllib.policy.policy import PolicySpec, Policy
from ray.rllib.utils.annotations import override
from ray.rllib.examples.policy.random_policy import RandomPolicy
from rl.envs.othello import OthelloEnv

2023-08-14 19:24:02,522	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-14 19:24:02,546	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


In [3]:
register_env("othello", lambda _: OthelloEnv({}))

In [4]:
import gymnasium as gym
action = gym.spaces.Box(-1, 1, shape=(1, 1), dtype=np.float32).sample()
type(action)

numpy.ndarray

In [5]:
from typing import List
import random


class OthelloRandomPolicy(RandomPolicy):
    @override(Policy)
    def compute_actions(self, obs_batch, **kwargs):
        actions: List[int] = [
            self._valid_random_action(obs) for obs in obs_batch
        ] 
        return actions, [], {}

    def _valid_random_action(self, obs) -> int:
        valid_actions: List[int] = []
        for row in range(obs.shape[0]):
            for col in range(obs.shape[1]):
                if self._is_valid_action(obs, row, col):
                    valid_actions.append(row * obs.shape[0] + col)

        return random.choice(valid_actions)

    def _is_valid_move(self, row: int, col: int) -> bool:
        if row < 0 or row > 7 or col < 0 or col > 7 or self.board[row][col] != 0:
            return False

        marks = {"agent_1": 1, "agent_2": -1, "empty": 0}
        agent_id = marks[self.__policy_id]
        directions = [
            (1, 0),
            (-1, 0),
            (0, 1),
            (0, -1),
            (1, 1),
            (1, -1),
            (-1, 1),
            (-1, -1),
        ]

        for d_row, d_col in directions:
            r, c = row + d_row, col + d_col
            if 0 <= r < 8 and 0 <= c < 8 and self.board[r][c] == -agent_id:
                r += d_row
                c += d_col
                while 0 <= r < 8 and 0 <= c < 8 and self.board[r][c] == -agent_id:
                    r += d_row
                    c += d_col
                if 0 <= r < 8 and 0 <= c < 8 and self.board[r][c] == agent_id:
                    return True

        return False

In [6]:
import os

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    agent_id = int(agent_id[-1]) - 1
    return "agent_1" if episode.episode_id % 2 == agent_id else "agent_2"

config = ppo.PPOConfig().environment("othello").framework("torch").rollouts(num_rollout_workers=os.cpu_count() // 2).resources(num_gpus=0)
config = config.multi_agent(policies={"agent_1": PolicySpec(), "agent_2": PolicySpec(policy_class=OthelloRandomPolicy)}, policy_mapping_fn=policy_mapping_fn, policies_to_train=["agent_1"])
config = config.training(model={"fcnet_hiddens": [512, 512]}, _enable_learner_api=False)
# config = config.training(num_sgd_iter=10, model={"conv_filters": [[32, [3, 3], 1], [64, [3, 3], 1]]})
# config = config.training(model={"conv_filters": [[32, [3, 3], 2], [64, [3, 3], 2]]}, _enable_learner_api=False)
# config = config.training(_enable_learner_api=False)
config = config.rl_module(_enable_rl_module_api=False)

2023-08-14 19:24:02,837	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-14 19:24:02,874	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


In [7]:
# import random
# # multiagent environment for othello
# env = OthelloEnv({})
# obs, _ = env.reset()
# terminated = False
# current_player = "agent_1"
# while not terminated:
#     env.render()
#     # action = env.action_space.sample()
#     valid_actions = env.get_valid_moves(current_player)
#     print(current_player, obs.keys())
#     print(obs[current_player].shape)
#     action = algo.compute_single_action(obs[current_player], policy_id=current_player)
#     print(action)
#     if len(valid_actions) > 0:
#         action = random.choice(valid_actions)
#     else:
#         action = 64
#     action = {current_player: action}
#     obs, reward, terminated, truncated, _= env.step(action)
#     reward = reward[current_player]
#     terminated = terminated[current_player]
#     truncated = truncated[current_player]
#     current_player = env.current_player
#     print(terminated, truncated, reward)

In [8]:
from ray import air
from ray import tune

results = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_frequency=50,
        )
    ),
).fit()

2023-08-14 19:24:25,970	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-14 19:24:25,974	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
(pid=1920) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1920) 2023-08-14 19:25:01,791	WARNING algorithm_config.p

In [ ]:
results

ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': 'fef35_00000'},
    path='c://\\Users\\lcglab\\ray_results\\IMPALA\\IMPALA_othello_fef35_00000_0_2023-08-11_00-17-18',
    checkpoint=None
  )
]>

(Impala pid=21228) 2023-08-11 00:17:38,050	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=16064, ip=127.0.0.1, actor_id=77c468a4531990bdcb4240c601000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000022313862700>)
(Impala pid=21228)   File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
(Impala pid=21228)   File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
(Impala pid=21228)   File "c:\Users\lcglab\miniconda3\envs\rl\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
(Impala pid=21228)     return method(__ray_actor, *args, **kwargs)
(Impala pid=21228)   File "c:\Users\lcglab\miniconda3\envs\rl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
(Impala pid=21228)     return method(self, *_args, **_kwargs)
(Impala pid=2

In [ ]:
checkpoint = results.get_best_result().checkpoint
print(checkpoint)

None


In [ ]:
import ray
ray.shutdown()
register_env("othello", lambda _: OthelloEnv({}))

In [ ]:
config.expolore = False
algo = config.build()
algo.restore(checkpoint)

2023-08-11 00:17:41,280	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
c:\Users\lcglab\miniconda3\envs\rl\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\lcglab\miniconda3\envs\rl\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could su

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [ ]:
algo.get_policy("agent_1").export_model("othello_policy", onnx=18)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import random
# multiagent environment for othello
env = OthelloEnv({})
obs, _ = env.reset()
terminated = False
current_player = "agent_1"
while not terminated:
    env.render()
    # action = env.action_space.sample()
    valid_actions = env.get_valid_moves(current_player)
    action = algo.compute_single_action(obs[current_player], policy_id=current_player)
    # if len(valid_actions) > 0:
    #     action = random.choice(valid_actions)
    # else:
    #     action = 64
    action = {current_player: action}
    print(action)
    obs, reward, terminated, truncated, _= env.step(action)
    reward = reward[current_player]
    terminated = terminated[current_player]
    truncated = truncated[current_player]
    current_player = env.current_player

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

{'agent_1': 29}
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

{'agent_2': 37}
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|X|X|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

{'agent_1': 19}


In [ ]:
env.render()

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|X|X|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

